# Data Anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage30yr""","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage30yr""","""Unknown""",669,"""Laurianneshire""","""AutoLoans""","""Accepted"""
"""MoneyMarketSavingsAccount""","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmieshire""","""DepositAccounts""","""Accepted"""
"""UPlusFinPersonal""","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccounts""","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
str,str,f64,str,str,str,bool
"""../../../../data/SampleHDS.jso…","""9290696907273502702""",2.8448e18,"""13273462532994275286""","""16138599964586145670""","""251058571276432204""",false
"""../../../../data/SampleHDS.jso…","""3501385184977768495""",9.3440e18,"""10131755847073285334""","""16138599964586145670""","""5827918498744954324""",true
"""../../../../data/SampleHDS.jso…","""18397638399460712938""",1.5701e19,"""2328768548294123391""","""4483231989430303041""","""251058571276432204""",false
"""../../../../data/SampleHDS.jso…","""3501385184977768495""",3.9332e18,"""6801397021320985502""","""16566081692875180502""","""251058571276432204""",false
"""../../../../data/SampleHDS.jso…","""9290696907273502702""",1.0603e18,"""2506018600311723107""","""16566081692875180502""","""3391530805412893797""",true
"""../../../../data/SampleHDS.jso…","""18397638399460712938""",7.1366e17,"""2875382368114983342""","""11227272905135642445""",null,false
"""../../../../data/SampleHDS.jso…","""18397638399460712938""",2.3074e18,"""3050298108943098080""","""16566081692875180502""","""3391530805412893797""",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'filename': 'filename',
 'Customer_MaritalStatus': 'PREDICTOR_1',
 'Customer_CLV': 'PREDICTOR_2',
 'Customer_City': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,f64,str,str,str,str
"""../../../../data/SampleHDS.jso…","""2192344891735682244""",4.1019e18,"""10372041764087666234""","""FirstMortgage30yr""","""13979833645192498843""","""Rejected"""
"""../../../../data/SampleHDS.jso…","""2636970147018559192""",1.4672e18,"""11627893521730451562""","""FirstMortgage30yr""","""13183962862427934764""","""Accepted"""
"""../../../../data/SampleHDS.jso…","""6881016052143307260""",9.2698e18,"""160305067431659914""","""MoneyMarketSavingsAccount""","""13979833645192498843""","""Rejected"""
"""../../../../data/SampleHDS.jso…","""2636970147018559192""",1.1843e19,"""17586717007926432265""","""BasicChecking""","""13979833645192498843""","""Rejected"""
"""../../../../data/SampleHDS.jso…","""2192344891735682244""",1.0469e19,"""3334632982831573700""","""BasicChecking""","""5239728048338093937""","""Accepted"""
"""../../../../data/SampleHDS.jso…","""6881016052143307260""",7.6051e18,"""10400199843330799781""","""UPlusFinPersonal""",null,"""Rejected"""
"""../../../../data/SampleHDS.jso…","""6881016052143307260""",5.9874e18,"""18000579479267922111""","""BasicChecking""","""5239728048338093937""","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,f64,str,str,str,str
"""../../../../data/SampleHDS.jso…","""16563416606604171344""",1.6169e19,"""5736459754255529832""","""FirstMortgage30yr""","""961799643224585796""","""Rejected"""
"""../../../../data/SampleHDS.jso…","""17492716056040859995""",1.3675e18,"""10297922878076795742""","""FirstMortgage30yr""","""11844090999704295929""","""Accepted"""
"""../../../../data/SampleHDS.jso…","""14627204154638799643""",1.0152e19,"""7515766168711231939""","""MoneyMarketSavingsAccount""","""961799643224585796""","""Rejected"""
"""../../../../data/SampleHDS.jso…","""17492716056040859995""",1.6680e19,"""1518912504615945380""","""BasicChecking""","""961799643224585796""","""Rejected"""
"""../../../../data/SampleHDS.jso…","""16563416606604171344""",1.0285e18,"""1647303866071534864""","""BasicChecking""","""15984657781101378302""","""Accepted"""
"""../../../../data/SampleHDS.jso…","""14627204154638799643""",2.4055e18,"""3704976643923706106""","""UPlusFinPersonal""",null,"""Rejected"""
"""../../../../data/SampleHDS.jso…","""14627204154638799643""",1.8263e19,"""853346816451868867""","""BasicChecking""","""15984657781101378302""","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

filename=filename
Customer_MaritalStatus=PREDICTOR_1
Customer_CLV=PREDICTOR_2
Customer_City=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str
"""12627307941091580418""",8.0235e18,"""25157198722377422""","""FirstMortgage30yr""","""4295164060856894422""","""Rejected"""
"""10935410098080431797""",8.0171e18,"""14136156570581473276""","""FirstMortgage30yr""","""1154972244858176757""","""Accepted"""
"""306828318694733161""",7.1946e18,"""11313055665305484622""","""MoneyMarketSavingsAccount""","""4295164060856894422""","""Rejected"""
"""10935410098080431797""",1.2790e19,"""18226948181616197301""","""BasicChecking""","""4295164060856894422""","""Rejected"""
"""12627307941091580418""",3.0393e18,"""11091739706092406376""","""BasicChecking""","""6884931819761883326""","""Accepted"""
"""306828318694733161""",8.3995e18,"""7221985708650499481""","""UPlusFinPersonal""",null,"""Rejected"""
"""306828318694733161""",9.7667e18,"""800498179031730829""","""BasicChecking""","""6884931819761883326""","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.



ComputeError: AttributeError: 'int' object has no attribute 'encode'